<a href="https://colab.research.google.com/github/nihalbaig0/dphi_notebooks/blob/master/P100_Instance_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copy this Notebook to always get a Colab Notebook with a Tesla P100 GPU and 25GB of RAM 

# Credits to 0x00groot for finding this
https://www.reddit.com/r/MachineLearning/comments/m73sy7/p_my_side_project_cloud_gpus_for_13_the_cost_of/gratmkg?utm_source=share&utm_medium=web2x&context=3

# Link to original notebook
https://colab.research.google.com/drive/1D6krVG0PPJR2Je9g5eN_2h6JP73_NUXz#scrollTo=W3a4PwlSfS8b

In [ ]:
!nvidia-smi

Thu Mar 18 19:03:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install comet_ml

     |████████████████████████████████| 256kB 5.9MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 522kB 34.6MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp37-none-any.whl size=34547 sha256=29d67e2d329a8bfd996aacabc5964488b67b035740382598595fe6f42852de53
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [ ]:
import comet_ml
import getpass, os
os.environ["COMET_API_KEY"] = getpass.getpass("Paste your COMET API KEY: ")

Paste your COMET API KEY: ··········


In [ ]:
from tensorflow import keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

LABELS = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]


class ConfusionMatrixCallback(keras.callbacks.Callback):
    def __init__(self, experiment, inputs, targets, interval):
        self.experiment = experiment

        self.inputs = inputs
        self.targets = targets
        self.interval = interval

    def index_to_example(self, index):
        image_array = self.inputs[index]
        image_name = "confusion-matrix-%05d.png" % index
        results = experiment.log_image(image_array, name=image_name)
        # Return sample, assetId (index is added automatically)
        return {"sample": image_name, "assetId": results["imageId"]}

    def on_epoch_end(self, epoch, logs={}):
        if (epoch + 1) % self.interval != 0:
            return

        predicted = self.model.predict(self.inputs)
        self.experiment.log_confusion_matrix(
            self.targets,
            predicted,
            labels=LABELS,
            index_to_example_function=self.index_to_example,
            title="Confusion Matrix, Epoch #%d" % (epoch + 1),
            file_name="confusion-matrix-%03d.json" % (epoch + 1),
        )


# Model configuration
batch_size = 8092
img_width, img_height, img_num_channels = 32, 32, 3
loss_function = categorical_crossentropy
no_classes = 10
no_epochs = 5
optimizer = Adam()
verbosity = 1
validation_split = 0.2
interval = 1

# Load CIFAR-100 data
(input_train, target_train), (input_test, target_test) = cifar10.load_data()

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

target_train, target_test = tuple(
    map(lambda x: keras.utils.to_categorical(x), [target_train, target_test])
)

# Parse numbers as floats
input_train = input_train.astype("float32")
input_test = input_test.astype("float32")

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Create the model
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu", input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(no_classes, activation="softmax"))

# Compile the model
model.compile(loss=loss_function, optimizer=optimizer, metrics=["accuracy"])

PROJECT_NAME = "p100-example"
experiment = comet_ml.Experiment(
    project_name=PROJECT_NAME,
    auto_metric_logging=True,
    auto_param_logging=True,
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
)
experiment.log_parameter("batch_size", batch_size)
confmat = ConfusionMatrixCallback(experiment, input_test, target_test, interval)

# Fit data to model
model.fit(
    input_train,
    target_train,
    batch_size=batch_size,
    epochs=no_epochs,
    verbose=verbosity,
    validation_split=validation_split,
    callbacks=[confmat],
)

In [ ]:
experiment.display()

In [ ]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/team-comet-ml/p100-example/ae31ba992f944a1da4bdf7eddbec93f8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [5]                : (0.12142500281333923, 0.29512500762939453)
COMET INFO:     batch_accuracy [5]          : (0.10998517274856567, 0.27570441365242004)
COMET INFO:     batch_loss [5]              : (1.958383560180664, 2.303607702255249)
COMET INFO:     epoch_duration [5]          : (20.23970210599964, 23.99688416800018)
COMET INFO:     loss [5]                    : (1.9239593744277954, 2.2899484634399414)
COMET INFO:     val_accuracy [5]            : (0.17190000414848328, 0.3156000077724457)
COMET INFO:     val_loss [5]                : (1.879258155822754, 2.2446868419647217)
COMET INFO:     validate_batch_accuracy [5] : 